In [1]:
# pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# conda install pytorch torchvision cudatoolkit=11.3 -c pytorch
import torch
import torch.nn as nn
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
from data import DataGenerator

data = DataGenerator()

In [3]:
class NCF(nn.Module):
    def __init__(self, num_users, num_likes, embed_size, num_hidden, output_size):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(num_users, embed_size)
        self.like_emb = nn.Embedding(num_likes,embed_size)
        self.fc1 = nn.Linear(embed_size*2, num_hidden[0])
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(num_hidden[0], num_hidden[1])
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(num_hidden[1], num_hidden[2])
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(num_hidden[2], num_hidden[3])
        self.relu4 = nn.ReLU()
        self.outLayer = nn.Linear(num_hidden[3], output_size)
        self.out_act = nn.Sigmoid()

    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.like_emb(v)
        out = torch.cat([U,V], dim=1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.outLayer(out)
        out = self.out_act(out)
        return out


In [4]:
emb_size = 128
hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1
num_epochs = 25
learning_rate = 0.001
batch_size = 2048
num_negatives = 5

random_samples = 100
top_K = 10

preTrained_NCF = NCF(data.num_users, data.num_movies, emb_size, hidden_layers, output_size).to(device)

In [5]:
'''SET MODEL TO TRAINING MODE'''
import torch.optim as optim

# set loss = BINARY CROSS ENTROPY
criterion = nn.BCELoss()
# ADAM Optimizer
optimizer = optim.Adam(preTrained_NCF.parameters(), lr=learning_rate, weight_decay=1e-6)
preTrained_NCF.train()

NCF(
  (user_emb): Embedding(6040, 128)
  (like_emb): Embedding(3952, 128)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (relu4): ReLU()
  (outLayer): Linear(in_features=16, out_features=1, bias=True)
  (out_act): Sigmoid()
)

In [6]:
'''TRAIN MODEL ( MINI BATCH )'''
num_batches = np.int64(np.floor(data.train.shape[0] / batch_size))
for i in range(num_epochs):
    j = 0
    for batch in np.array_split(data.train, num_batches):
        batch_df = data.add_negatives(batch[['uid', 'mid', 'rating']], n_samples=num_negatives)
        users = torch.LongTensor(batch_df.uid.to_numpy()).to(device)
        items = torch.LongTensor(batch_df.mid.to_numpy()).to(device)
        ratings = torch.FloatTensor(batch_df.rating.to_numpy()).to(device)
        y_hat = preTrained_NCF(users, items)
        loss = criterion(y_hat, ratings.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(f'epoch: {i + 1} \nbatch: {j} out of: {num_batches} \naverage loss: {loss.item()}\n')
        # j += 1


In [14]:
data.train[data.train.uid==4].mid.sort_values(ascending=True)

297       5
387      15
421      23
426      28
442      31
       ... 
295    3727
345    3743
446    3785
276    3792
281    3798
Name: mid, Length: 197, dtype: int64

In [ ]:
def train():
    train = data.add_negatives(data.train[['uid', 'mid', 'rating']], n_samples=4)
    users = torch.LongTensor(train.uid.to_numpy()).to(device)
    items = torch.LongTensor(train.mid.to_numpy()).to(device)
    ratings = torch.FloatTensor(train.rating.to_numpy()).to(device).unsqueeze(1)

    for i in range(num_epochs):
        y_hat = preTrained_NCF(users, items)
        loss = criterion(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'loss: {loss}')

In [49]:
import pandas as pd

def evaluate_model(model, df_val:pd.DataFrame, top_K, random_samples):
    model.eval()
    avg_HR = np.zeros((len(df_val), top_K))
    avg_NDCG = np.zeros((len(df_val), top_K))
    test_df = data.add_negatives(
        df_val,
        n_samples=random_samples
    )
    gp = test_df.groupby('uid')
    for g in gp:
        for k in range(top_K):
            users, items = torch.LongTensor(g.uid).to(device), torch.LongTensor(g.mid).to(device)
            y_hat = model(users, items)
            y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
            test_item_input = items.cpu().detach().numpy().reshape((-1,))
            map_item_score = dict(zip(test_item_input, y_hat))

    for i in range(df_val.shape[0]):
        for k in range(top_K):
            test_df = data.add_negatives(
                pd.DataFrame(data.test.iloc[i]).T,
                n_samples=random_samples
            )
            users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.mid).to(device)
            y_hat = model(users, items)
            y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
            test_item_input = items.cpu().detach().numpy().reshape((-1,))
            map_item_score = dict(zip(test_item_input, y_hat))
            # Evaluate top rank list
            ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
            gtItem = test_item_input[0]
            for item in ranklist:
                if item==gtItem:
                    avg_HR[i, k] = 1
                    avg_NDCG[i, k] = math.log(2) / math.log(i+2)
                else:
                    avg_HR[i, k] = 0
                    avg_NDCG[i, k] = 0
    avg_HR = np.mean(avg_HR, axis=0)
    avg_NDCG = np.mean(avg_NDCG, axis=0)
    return avg_HR, avg_NDCG

def evaluate(model, df_val:pd.DataFrame, k=10):
    test_df = data.add_negatives(df_val, n_samples=random_samples)
    users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.mid).to(device)
    y_hat = model(users, items)
    test_df['score'] = y_hat.detach.numpy().reshape((-1,))
    grouped = test_df.copy(deep=True)
    grouped['ranked'] = grouped.groupby('uid')['score'].rank(method='first', ascending=False)
    grouped.sort_values(['uid', 'rank'], inplace=True)
    top_k = grouped[grouped['rank']<=k]
    test_in_top_k = top_k[top_k['rating'] == 1]
    hr = test_in_top_k.shape[0] / data.num_users
    test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: np.log(2)/np.log(1 + x))
    ndcg = test_in_top_k.ndcg.sum() / data.num_users
    return hr, ndcg


In [7]:
torch.save(preTrained_NCF.state_dict(), "models/preTrained_NCF")




In [ ]:
# debiased_NCF = NCF(data.num_users, data.num_movies, emb_size, hidden_layers, output_size).to(device)
# debiased_NCF.load_state_dict(torch.load("trained-models/preTrained_NCF"))
# debiased_NCF.to(device)

In [ ]:
import constants
import pandas as pd


def features():
    df = pd.read_csv('MovieLens/users.dat',
                     sep='::',
                     header=None,
                     names=['uid', 'gender', 'age', 'job', 'zip'],
                     engine='python')
    df.drop(columns=['uid'], inplace=True)
    df.index.rename('uid', inplace=True)
    df.gender = pd.get_dummies(df.gender, drop_first=True)  # 0:F, 1:M
    return df.reset_index()


features = features()

# complete = pd.merge(df, data.df, how=' outer', on='uid')
# complete.drop(columns=['date', 'latest', 'zip'], inplace=True)
# gender_embed = compute_gender_direction(train_data, train_protected_attributes, users_embed)
# S = 0 indicates male and S = 1 indicates female

In [ ]:
# see constants for more info
drop = [0, 10, 13, 19]

clean = features[~features['job'].isin(drop)]

clean['rating'] = 1
num_users = clean.uid.nunique()
# num_movies = clean.mid.nunique()
num_jobs = clean.job.nunique()

new_job_index = np.arange(num_jobs)
item_id = clean[['job']].drop_duplicates()
item_id['njob'] = np.arange(num_jobs)
clean = pd.merge(clean, item_id, on=['job'], how='left')
clean.job = clean.njob
clean.head()

In [ ]:
msk = np.random.rand(len(clean)) < 0.7

train = clean[msk]
test = clean[~msk]

clean.info()  # 1476+3444 = 4920

In [ ]:
''' GET USER EMBEDDING '''
user_embeds = preTrained_NCF.user_emb.weight.data.cpu().detach().numpy()
user_embeds = user_embeds.astype('float')

In [ ]:
''' COMPUTE GENDER EMBEDDING '''
gender_embed = np.zeros((2,user_embeds.shape[1]))
num_users_x_group = np.zeros((2, 1))

for i in range(train.shape[0]):
    u = train['uid'].iloc[i]
    if train['gender'].iloc[i] == 0:
        gender_embed[0] +=  user_embeds[u]
        num_users_x_group[0] += 1.0
    else:
        gender_embed[1] +=  user_embeds[u]
        gender_embed[1] += 1.0
        num_users_x_group[1] += 1.0

In [ ]:
''' VERTICAL BIAS'''
gender_embed = gender_embed / num_users_x_group
# vBias = compute_bias_direction(gender_embed)
vBias = gender_embed[1].reshape((1,-1)) - gender_embed[0].reshape((1,-1))
vBias = vBias / np.linalg.norm(vBias,axis=1,keepdims=1)

vBias

In [ ]:
''' LINEAR PROJECTION '''
debiased_user_embeds = user_embeds


for i in range(len(clean)):
    u = clean['uid'].iloc[i]
    debiased_user_embeds[u] = user_embeds[u] - (np.inner(user_embeds[u].reshape(1,-1),vBias)[0][0])*vBias


In [ ]:
'''UPDATE USER EMBEDDINGS'''
fairness_thres = torch.tensor(0.1).to(device)
epsilonBase = torch.tensor(0.0).to(device)

n_careers = clean.job.nunique()
# replace page items with career items
preTrained_NCF.like_emb = nn.Embedding(n_careers,emb_size).to(device)
# freeze user embedding
preTrained_NCF.user_emb.weight.requires_grad=False

# replace user embedding of the model with debiased embeddings
preTrained_NCF.user_emb.weight.data = torch.from_numpy(debiased_user_embeds.astype(np.float32)).to(device)


In [ ]:
'''OPTIMIZE'''
# fair_fine_tune_model(DF_NCF,train_data, num_epochs, learning_rate,batch_size,num_negatives,n_careers,train_gender,fairness_thres,epsilonBase, unsqueeze=True)
emb_size = 128
num_epochs = 10
batch_size = 256

num_negatives = 5

random_samples = 15
top_k = 10

criterion = nn.BCELoss()
optimizer = optim.Adam(preTrained_NCF.parameters(), lr=learning_rate, weight_decay=1e-6)

preTrained_NCF.train()

torch.save(preTrained_NCF.state_dict(), "models/DF_NCF")

In [ ]:
'''FAIR FINE TUNING MODEL'''
all_users = torch.LongTensor(train['uid'].values).to(device)
all_items = torch.LongTensor(train['job'].values).to(device)
all_genders = torch.LongTensor(train['gender'].values).to(device)

from fairness_measures import Measures

m = Measures()
num_batches = np.int64(np.floor(train.shape[0] / batch_size))

for i in range(num_epochs):
    j = 1
    for batch in np.array_split(train, num_batches):
        batch_df = data.add_negatives(
            df=batch[['uid', 'job', 'rating']],
            item='job',
            items=set(clean.job.unique()),
            n_samples=4
        )
        print(batch_df.head())
        users = torch.LongTensor(batch_df.uid.to_numpy()).to(device)
        items = torch.LongTensor(batch_df.job.to_numpy()).to(device)
        ratings = torch.FloatTensor(batch_df.rating.to_numpy()).to(device)
        print(items)
        y_hat = preTrained_NCF(users, items)

        loss1 = criterion(y_hat, ratings.unsqueeze(1))

        predicted_probs = preTrained_NCF(all_users, all_items)
        avg_epsilon = m.computeEDF(all_genders,predicted_probs,n_careers,all_items,device)
        print(avg_epsilon)
        #criteroin hinge
        loss2 = torch.max(torch.tensor(0.0).to(device), (avg_epsilon-epsilonBase))

        loss = loss1 + fairness_thres*loss2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'epoch: {i + 1} \nbatch: {j} out of: {num_batches} \naverage loss: {loss.item()}\n')
        j+=1

In [ ]:
torch.save(preTrained_NCF.state_dict(), "models/DF_NCF")

In [ ]:
import math
import heapq
def evaluate_fine_tune(model,df_val,top_K,random_samples):
    model.eval()
    avg_HR = np.zeros((len(df_val),top_K))
    avg_NDCG = np.zeros((len(df_val),top_K))

    # for i in range(len(df_val)):
    test_df = data.add_negatives(
        df_val,
        item='job',
        items=set(clean.job.unique()),
        n_samples=random_samples
    )
    users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.job).to(device)
    y_hat = model(users, items)

    y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
    items = items.cpu().detach().numpy().reshape((-1,))
    map_item_score = {}
    for j in range(len(y_hat)):
        map_item_score[items[j]] = y_hat[j]
    for k in range(top_K):
        # Evaluate top rank list
        ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
        gtItem = items[0]
        avg_HR[i,k] = getHitRatio(ranklist, gtItem)
        avg_NDCG[i,k] = getNDCG(ranklist, gtItem)
    avg_HR = np.mean(avg_HR, axis = 0)
    avg_NDCG = np.mean(avg_NDCG, axis = 0)
    return avg_HR, avg_NDCG

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [ ]:
'''EVALUATE TUNED MODEL'''
avg_HR_DF_NCF, avg_NDCG_DF_NCF = evaluate_fine_tune(preTrained_NCF, test, top_K, random_samples)

avg_HR_DF_NCF

In [ ]:
'''MEASURE THE FAIRNESS OF THE MODEL'''
def fairness_measures(model,df_val,num_items):
    model.eval()
    users, items = torch.LongTensor(df_val.uid.to_numpy()).to(device), torch.LongTensor(df_val.job.to_numpy()).to(device)
    y_hat = model(users, items)

    avg_epsilon = m.computeEDF(all_genders.cpu(),y_hat,num_items,items,device)
    U_abs = m.compute_absolute_unfairness(all_genders.cpu(),y_hat,num_items,items,device)

    avg_epsilon = avg_epsilon.cpu().detach().numpy().reshape((-1,)).item()
    print(f"average differential fairness: {avg_epsilon: .3f}")

    U_abs = U_abs.cpu().detach().numpy().reshape((-1,)).item()
    print(f"absolute unfairness: {U_abs: .3f}")

fairness_measures(preTrained_NCF,test,n_careers)


